In [ ]:
#Here we load our contacts data in the text bedpe format (.cool dumped data is bedpe)
import pandas as pd
contacts = pd.read_table('Your bedpe dataset.txt', sep=r'\t', engine='python', names=['chrom1_name', 'chrom1_start_locus','chrom1_end_locus', 'chrom2_name', 'chrom2_start_locus','chrom2_end_locus','number_of_contacts'], index_col=False)
contacts.head() #check that all is good

In [ ]:
#Here we get the distances as inverted frequences
alpha = 0.1 #choose your float alpha -- power of inversion
contacts['distance_alpha={}'.format(alpha)] = contacts['number_of_contacts'].apply(lambda x: 1/(x**alpha) if x != 0 else 2.0)
contacts.head() #check that all is good

In [ ]:
#Let's explore inTERchromosomal contacts
#See what is the sum number of contacts between different chromosomes:
interchrom_contacts = contacts.groupby(['chrom1_name', 'chrom2_name'], as_index=False)[['number_of_contacts']].sum()
interchrom_contacts.head()

In [ ]:
chosen_chromosome ='2' #choose number of a chromosome instead of '1'
#You'd better not change the name of the following dataframe (it's used everywhere below):
chromosome_1_contacts = interchrom_contacts[(interchrom_contacts['chrom1_name'] == chosen_chromosome)&(interchrom_contacts['chrom2_name'] != chosen_chromosome)]
#Now we can see with what chromosome your chosen chromosome interacts most...
chromosome_1_contacts[chromosome_1_contacts['number_of_contacts']==chromosome_1_contacts['number_of_contacts'].max()]

In [ ]:
#...And with what -- least
chromosome_1_contacts[chromosome_1_contacts['number_of_contacts']==chromosome_1_contacts['number_of_contacts'].min()]

In [ ]:
#Or you can see the full spectre:
chromosome_1_contacts.sort_values(by='number_of_contacts')

In [ ]:
#Let's explore inTRAchromosomal contacts
intrachrom_contacts = contacts[(contacts['chrom1_name'] == chosen_chromosome)&(contacts['chrom2_name'] == chosen_chromosome)]
intrachrom_contacts = intrachrom_contacts[['chrom1_start_locus','chrom2_start_locus','number_of_contacts']]
intrachrom_contacts.head()

In [ ]:
#Let's see what two loci interacts the most
intrachrom_contacts[intrachrom_contacts['number_of_contacts']==intrachrom_contacts['number_of_contacts'].max()]

In [ ]:
#Let's see what two loci interacts the least
intrachrom_contacts[intrachrom_contacts['number_of_contacts']==intrachrom_contacts['number_of_contacts'].min()]

In [ ]:
#And the full spectre
intrachrom_contacts.sort_values(by='number_of_contacts')

In [ ]:
#Here the chromosoms lengths are
chrom_lengths = contacts.groupby('chrom1_name', as_index=False)[['chrom1_end_locus']].max() 
chrom_lengths.columns = ['chromosome', 'length']
chrom_lengths #check that all is good

In [ ]:
#Let's compute and compare the degree of inner interactions of the chromosomes
#To do this we divide the number of intrachromosomal interactions by their length
 
only_intra_contacts = contacts[contacts['chrom1_name']==contacts['chrom2_name']]
chromosomes_entanglements = only_intra_contacts.groupby('chrom1_name',as_index=False)[['number_of_contacts']].sum()
chromosomes_entanglements['entanglement'] = chromosomes_entanglements['number_of_contacts']/chrom_lengths['length']
chromosomes_entanglements.columns = ('chromosome','contacts','entanglement')
chromosomes_entanglements.sort_values(by='entanglement')

In [ ]:
#Sort:
chromosomes_entanglements.sort_values(by='entanglement')

In [ ]:
#Also we can compare the normalized means of intrachromosomal interactions
#Firstly, we apply min-max normalisation to mean value: compute min, max and mean...
only_intra_contacts['average'] = only_intra_contacts.groupby('chrom1_name', as_index=False)['number_of_contacts'].transform('mean')
only_intra_contacts['minimum'] =  only_intra_contacts.groupby('chrom1_name', as_index=False)['number_of_contacts'].transform('min')
only_intra_contacts['maximum'] =  only_intra_contacts.groupby('chrom1_name', as_index=False)['number_of_contacts'].transform('max')

In [ ]:
#...And apply
only_intra_contacts['normised_average'] = (only_intra_contacts['average']-only_intra_contacts['minimum'])/(only_intra_contacts['maximum']-only_intra_contacts['minimum'])

In [ ]:
#Here we create a dataframe with alter entanglement -- normalised mean
chromosomes_alter_entanglements = only_intra_contacts.groupby('chrom1_name',as_index=False)[['normised_average']].max()
chromosomes_alter_entanglements.columns = ['chromosome','alter_entanglement']
chromosomes_alter_entanglements.sort_values(by='alter_entanglement')

In [ ]:
#Let's compare entanglements:
chromosomes_entanglements.sort_values(by='chromosome')
chromosomes_alter_entanglements.sort_values(by='chromosome')
chrom_lengths.sort_values(by='chromosome')
comparing_entanglements = chromosomes_entanglements.join([chromosomes_alter_entanglements['alter_entanglement'],chrom_lengths['length']])
comparing_entanglements

In [ ]:
#Let's see how columns Spearman-correlate:
comparing_entanglements.corr(method='spearman')

In [ ]:
#And how columns Kendall-correlate:
comparing_entanglements.corr(method='kendall')

In [ ]:
#Let's visualise to see (or not) correlation:
points_cloud_ents = comparing_entanglements.plot.scatter(x='entanglement',y='alter_entanglement')

In [ ]:
points_cloud_ent_lengths = comparing_entanglements.plot.scatter(x='entanglement',y='length')

In [ ]:
points_cloud_altent_lengths = comparing_entanglements.plot.scatter(x='alter_entanglement',y='length')

In [ ]:
points_cloud_contacts_lengths = comparing_entanglements.plot.scatter(x='contacts',y='length')

In [ ]:
#Firstly, let's try linear regression
#If it doesn't fit -- non-linear curve fitting will be tried
from scipy.optimize import leastsq
import numpy as np
import matplotlib.pyplot as plt


def curve_fitting():
   # data provided
   x=np.array(comparing_entanglements['contacts'].tolist())
   y=np.array(comparing_entanglements['length'].tolist())
   # here, create lambda functions for  fit -- here linear
   # coefficients is a tuple that contains the parameters of the fit -- function coeffs
   Linear=lambda tpl,x : tpl[0]*x + tpl[1]
   
   # function we would like to fit
   func=Linear
   # ErrorFunc is the diference between the func and the y "experimental" data
   ErrorFunc=lambda tpl,x,y: func(tpl,x)-y
   #tplInitial contains the "first guess" of the parameters 
   tplInitial1=(1.0,-0.2)
   # leastsq finds the set of parameters in the tuple tpl that minimizes ErrorFunc
   
   tplFinal1,success=leastsq(ErrorFunc,tplInitial1[:],args=(x,y))
   print (" linear fit  ",tplFinal1)
   
   yy1=func(tplFinal1,x)
   #------------------------------------------------
   # now the quadratic fit
   #-------------------------------------------------
   

   
   plt.plot(x,yy1,'r--',x,y,'bo')
   plt.show()

curve_fitting()

In [ ]:
#Let's make new dataframe -- genome square with distances
#Firstly, we make indexes (there are also columns as it is a square)
#Indexes are chromosome bins in the format "x_y", x for the chromosome number, y for bin number 

res = 1000 #here is the resolution of your Hi-C data, integer
indexes = []
i = 0
for k in chrom_lengths['chromosome']:
    for j in range((chrom_lengths.loc[i]['length'])//res):
        indexes.append(k + '_' + str(j+1))
    i+=1
indexes.head() #check that all is good

In [ ]:
#Here we create a pandas DataFrame with chromosomes bins as indexes and columns
#if dataset is to big you may make a rectangle instead of a square -- diminush the columns number -- choose your diminishing coefficient, integer

dc = 1000 #here is the diminishing coefficient 
distances = pd.DataFrame(index=indexes, columns=indexes[:len(indexes)//dc])
for k in indexes:
    for m in indexes:
        try:
            distances.loc[k][m] = contacts[ (contacts['chrom2_end_locus']//res == int(m[m.find('_') + 1:]))&(contacts['chrom2_name'] == m[:m.index('_')]) &(contacts['chrom1_name'] == k[:k.index('_')]) & (contacts['chrom1_end_locus']//res == int(k[k.find('_') + 1:]))]['distance_alpha=0.1'].iat[0]
            break
        except IndexError:
            distances.loc[k][m] = 2.0

In [ ]:
#We need to get read of all NaNs in order to visualize our data
distances.fillna(value=2.0, inplace=True)

In [ ]:
#Check that all is good
distances.head()

In [ ]:
#Let's visualize our data as a heatmap
import seaborn as sns
sns.heatmap(distances, vmin=0, vmax=2.0, center=0.5, ) #parameteres in the brackets can be tuned differently
